# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

### Q3. 아래의 코드에 주석 달기.

In [ ]:
#Pytorch 라이브러리 import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

#GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)     #재현성을 위한 시드 고정
print(device + " is available")

In [ ]:
#학습 파라미터 설정
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
#Torchvision에서 제공하는 dataset 가져오기
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([  #(H,W,C) 형태의 이미지를 (C,H,W) 형태의 텐서로 바꿔주는 transform
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
#Pytorch에서 제공하는 DataLoader를 통해 간단하게 데이터를 병렬처리
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
#Model define
class ConvNet(nn.Module): #Pytorch에서는 nn.Module을 상속받아 모델 생성
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)        #CNN
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)   #Dropout
        self.mp = nn.MaxPool2d(2)                           #Pooling
        self.fc1 = nn.Linear(320,100)                       #MLP
        self.fc2 = nn.Linear(100,10) 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:

model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)      #손실함수 정의
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) #Adam optimizer 사용

In [ ]:

for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        #각 데이터를 device(gpu)로 보냄
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()                   #해당 epoch의 학습 전 optimizer의 gradient를 초기화해줌
        hypothesis = model(data)                #data를 model에 입력시켜 forward 연산
        cost = criterion(hypothesis, target)    #hypothesis와 실제 target값과의 차이를 바탕으로 CrossEntropy 연산
        cost.backward()                         #Loss를 Backprop하여 gradient 생성
        optimizer.step()                        #생성된 gradient를 바탕으로 parameter 업데이트
        avg_cost += cost / len(train_loader)    #학습 진행을 확인하기 위한 avg_cost 계산 및 출력
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:

model.eval()        #모델을 eval모드로 변경, (BatchNorm, dropout등이 train과 eval에서 다르게 작동)
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        #test data를 모델에 입력해 forward연산을 수행, train이 아니기 때문에 backprop은 진행하지 않음
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
    #총 Accuracy 계산 및 출력
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.